One off script to take a dozen pickle files of usage data and turn them into one file

In [1]:


import pickle
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
from collections import defaultdict

import sys
sys.path.append("..")
from clustering import make_usage_matrices


/home/gsc685/.conda/envs/gabriella_cwr4lsc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
decades = [decade for decade in np.arange(1910, 2009, 10)]
out_path = '../data/cwr4lsc/feature_prediction/usages_16_len128.dict'

all_usages = defaultdict(list)

for decade in decades:
    path = '../data/cwr4lsc/feature_prediction/usages_with_vectors_16_len128_{}.dict'.format(decade)
    with open(path, 'rb') as f:
        decade = pickle.load(f)
        
        #print(decade['leaf'])[0]

        # remove annoying initial (word, vector) tuple to just vector
        for target in decade.keys():
            decade[target] = [(tuplo[1], x, y, z) for tuplo, x, y, z in decade[target] ] 
            #decade[target] = (big_tuple[0][1], big_tuple[1], big_tuple[2], big_tuple[3]) for big_tuple in decade[target]


        
        #raise Exception('not done')

        for target in decade.keys():
            all_usages[target] += (decade[target])

In [3]:
len(all_usages['parent'])

6302

In [4]:
all_usages['leaf'][0]

(array([ 0.02825611,  0.01642441,  0.16426327, ...,  0.00230723,
        -0.00927058, -0.00955612]),
 ['to',
  'the',
  'land',
  'of',
  'israel',
  ',',
  'to',
  'st',
  '##ori',
  '##ed',
  'galilee',
  '.',
  'one',
  'swift',
  'line',
  'tells',
  'all',
  'his',
  'youth',
  '?',
  '"',
  'ii',
  '##e',
  'grew',
  'in',
  'grace',
  'and',
  'stature',
  ',',
  'in',
  'favor',
  'with',
  'god',
  'and',
  'man',
  '.',
  '"',
  'so',
  'that',
  'white',
  'childhood',
  'is',
  'swept',
  'into',
  'the',
  'innocent',
  'silence',
  'of',
  'all',
  'childhood',
  '.',
  'we',
  'think',
  'of',
  'it',
  'as',
  'going',
  'lightly',
  ',',
  'like',
  'a',
  'rose',
  '-',
  'leaf',
  'dancing',
  'on',
  'the',
  'shining',
  'floor',
  'of',
  'a',
  'river',
  '.',
  'the',
  'growing',
  'boy',
  'spent',
  'beautiful',
  'years',
  'at',
  'nazareth',
  ',',
  'a',
  'little',
  'hill',
  '-',
  'nest',
  '##ed',
  'gal',
  '##ile',
  '##an',
  'village',
  ',',
  '

In [5]:
# save into another file with all usages, in the format tht the clusterizer expects


with open(out_path, 'wb') as f:
    pickle.dump(all_usages, f)

now we have the right number of things to unpack in a single usage.

In [6]:
len(all_usages['leaf'][0])

4

In [7]:
with open(out_path, 'rb') as f:
    usages_test = pickle.load(f)

In [8]:
for w in usages_test:
    _, _, _, _ = usages_test

ValueError: too many values to unpack (expected 4)

actually it still doesn't work. the function doesn't want a llist of individual usages for a word, it wants a matrix for each word

We have to make udsage matrices actually which is what the clusterizer really expects. there's a utility for this already but we have to do it ourselves

In [9]:
in_path = '../data/cwr4lsc/feature_prediction/usages_16_len128.dict'
out_path = '../data/cwr4lsc/feature_prediction/matrix_usages_16_len128.dict'

matrix_usages = make_usage_matrices(in_path, usages_out=None, ndims=3981) # make sure to use the ndims of the feature prediction model
with open(out_path, 'wb') as f:
    pickle.dump(matrix_usages, f)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [1:42:33<00:00, 341.85s/it]
